In [3]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException


# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")


# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_games.html"
driver.get(url)

time.sleep(2)

## Final Scores

In [42]:
# Find months to filter data from
month_filter = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]')))

# Get link to each month
months = month_filter.find_elements(By.TAG_NAME, 'a')

current_date = datetime.now().strftime('%a, %b %d, %Y')


# Find the table on the page
table = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="schedule"]'))
)

# Get table headers
thead = table.find_element(By.TAG_NAME, 'thead')

# Get the table body
tbody = table.find_elements(By.TAG_NAME, 'tbody')


# Get data from each row in the table
rows = [tr.find_elements(By.TAG_NAME, 'tr') for tr in tbody][0]

if current_date == rows[0].text[:17]:
    print('yes')
else:
    print('no')



no


In [36]:
rows[0].text[:17]


'Tue, Oct 24, 2023'

In [43]:

# Find months to filter data from
month_filter = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]')))

# Get link to each month
months = month_filter.find_elements(By.TAG_NAME, 'a')

# Get current date
current_date = datetime.now().strftime('%b %d, %Y')
current_date_found = False
table_data = []
for i in range(len(months)):
    # Re-locate the element with the months to avoid stale element reference
    month_filter = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]')))
    months = month_filter.find_elements(By.TAG_NAME, 'a')
    
    # Get data for each month
    month = months[i].text
    print(f"Gathering data from {month}")
    
    # Change month to lowercase 
    month = months[i].text.lower()
    
    # Loop through all months
    url = f"https://www.basketball-reference.com/leagues/NBA_2024_games-{month}.html"
    driver.get(url)
    
    # Find the table on the page
    table = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="schedule"]'))
    )

    # Get table headers
    thead = table.find_element(By.TAG_NAME, 'thead')

    # Get the table body
    tbody = table.find_elements(By.TAG_NAME, 'tbody')


    # Get data from each row in the table
    rows = [tr.find_elements(By.TAG_NAME, 'tr') for tr in tbody][0]
    for row in rows:
        # Print the final scores on each date
        if current_date == rows[0].text[:17]:
            print(row.text)
            table_data.append(row.text)
        else:
            current_date_found = True
            break

Gathering data from October


WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: MicrosoftEdge=120.0.2210.91)


## Box Scores

In [81]:
# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_games.html"
driver.get(url)

# Find the table on the page
table = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="schedule"]'))
)

# Find all elements with an 'a' tag
links = table.find_elements(By.TAG_NAME, 'a')

# Find href links that contain the boxscores
box_scores = [link.get_attribute('href') for link in links if '.html' in link.get_attribute('href') and '/boxscores/' in link.get_attribute('href') ]

In [122]:
for score in box_scores:
    driver.get(score)
    
    # Find the line score table
    line_score_table = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="line_score"]'))
                                                     )
    
    # Get all quarters and total score from table
    line_score_headers = line_score_table.find_elements(By.TAG_NAME, 'tr')[1].text.split(' ')
    
    # Modify headers
    line_score_headers[1] = 'team'
    line_score_headers[-1] = 'total'
    line_score_headers.pop(0)

    # Table body contains the teams, their respective scores per quarter and final score
    # tbody = line_score_table.find_element(By.TAG_NAME, 'tbody')
    # table_rows = tbody.find_elements(By.TAG_NAME, 'tr')

    # teams = []
    # for row in table_rows:
    #     row = row.text
    #     # Team abbreviation
    #     teams.append(row[:3])

    # Get team abbreviation quarter points and final game points
    team_final_points = line_score_table.find_elements(By.TAG_NAME, 'tr')[2:]
    final_points = []
    
    for points in team_final_points:
        final_points.append(points.text.split(' '))

# Create line score dataframe
line_score_df = pd.DataFrame(final_points, columns=line_score_headers)
line_score_df

    # # Home team and Away team
    # away_team = teams[0]
    # home_team = teams[1]

KeyboardInterrupt: 

## Basic Stats

In [ ]:
teams

['LAL', 'DEN', 'PHO', 'GSW', 'HOU', 'ORL']

In [ ]:
stats = ['basic', 'advanced']
basic_table_stats = []
advanced_table_stats = []

for stat in stats:
    if stat == 'basic':
        # Find basic stat table
        basic_table = driver.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]')

        # Get headers
        table_headers = basic_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/thead/tr[2]').text

        # Get data form table
        table_body = basic_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/tbody')
        table_rows = table_body.text.split('\n')
        
        basic_table_stats.append(table_rows)
        
    else:
        # Find advanced stat table
        advanced_table = driver.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]')

        # Get headers
        table_headers = advanced_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/thead/tr[2]').text

        # Get data form table
        table_body = advanced_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/tbody')
        table_rows = table_body.text.split('\n')
        
        advanced_table_stats.append(table_rows)


basic_table_stats = basic_table_stats[0]
advanced_table_stats = advanced_table_stats[0]

basic_table_stats

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: MicrosoftEdge=120.0.2210.91)


In [ ]:
stats = ['basic', 'advanced']
basic_table_stats = []
advanced_table_stats = []

for stat in stats:
    if stat == 'basic':
        # Find basic stat table
        basic_table = driver.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]')

        # Get headers
        table_headers = basic_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/thead/tr[2]').text

        # Get data form table
        table_body = basic_table.find_element(By.XPATH, f'//*[@id="box-{teams[0]}-game-{stat}"]/tbody')
        table_rows = table_body.text.split('\n')
        
        basic_table_stats.append(table_rows)
        
basic_table_stats = basic_table_stats[0]
basic_table_stats

["D'Angelo Russell 36:11 4 12 .333 2 5 .400 1 2 .500 0 4 4 7 1 0 3 3 11 +1",
 'Anthony Davis 34:09 6 17 .353 1 2 .500 4 4 1.000 1 7 8 4 0 2 2 3 17 -17',
 'Austin Reaves 31:20 4 11 .364 1 2 .500 5 7 .714 4 4 8 4 2 0 2 2 14 -14',
 'Taurean Prince 29:53 6 8 .750 4 6 .667 2 2 1.000 1 2 3 1 0 1 1 0 18 -14',
 'LeBron James 29:00 10 16 .625 1 4 .250 0 1 .000 1 7 8 5 1 0 0 1 21 +7',
 'Reserves MP FG FGA FG% 3P 3PA 3P% FT FTA FT% ORB DRB TRB AST STL BLK TOV PF PTS +/-',
 'Gabe Vincent 22:18 3 8 .375 0 4 .000 0 0 1 0 1 2 1 0 2 3 6 -17',
 'Cam Reddish 17:38 2 4 .500 1 2 .500 2 2 1.000 2 2 4 0 0 1 0 2 7 +7',
 'Christian Wood 15:28 3 4 .750 0 1 .000 1 2 .500 1 3 4 0 0 0 1 1 7 +2',
 'Rui Hachimura 14:39 3 10 .300 0 3 .000 0 0 2 1 3 0 0 0 0 2 6 -8',
 'Jaxson Hayes 6:54 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 -7',
 'Max Christie 1:15 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0',
 'Maxwell Lewis 1:15 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0']